## Ejemplo de Pendulum

In [ ]:
# env = gym.make("Pendulum-v1", render_mode="human")
# observation, info = env.reset()

# for _ in range(250):
#     action = env.action_space.sample()  # agent policy that uses the observation and info
#     observation, reward, terminated, truncated, info = env.step(action)

#     print(observation, reward, terminated, truncated, info)

#     if terminated or truncated:
#         observation, info = env.reset()

# env.close()

## Basic

In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [2]:
#env = gym.make("Pendulum-v1", render_mode="rgb_array", g=9.81)
env = gym.make("Pendulum-v1", render_mode="human", g=9.81)


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.cuda.is_available())

# ********************************************************************

def calculate_reward(observ, torque, target_angle): # Todos los valores estan en radianes
    #theta, theta_dot = state
    theta = math.atan2(observ[1],observ[0])
    theta_dot = observ[2]
    
    theta_n = ((theta + np.pi) % (2*np.pi)) - np.pi

    theta_error = np.abs(theta_n - target_angle)
    
    #costs = (theta_error**2) + 0.1 * (theta_dot**2) + 0.1 * (torque**2)

    torque_castigo = (torque**2) - np.minimum(2-np.absolute(torque),0)
    costs = (theta_error**2) + 0.1 * (theta_dot**2) + torque_castigo

    if theta_error <= 0.26: # ~ 15°
        reward_n = -costs + math.exp(-(8*theta_error)**2)
    else:
        reward_n = -costs
    #reward_n = -costs

    return torch.tensor([reward_n], device=device)


True


## Replay Memory

In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

## DQN Algorithm

### Q-network

In [4]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.active1 = nn.PReLU(128, random.uniform(0.1, 0.5))    ##
        self.layer2 = nn.Linear(128, 128)
        self.active2 = nn.PReLU(128, random.uniform(0.1, 0.5))    ##
        self.output_layer = nn.Linear(128, n_actions)
    def forward(self, x):
        #x = F.prelu(self.norm1(self.layer1(x)), 0.5*torch.rand(1, dtype=torch.float32, device=device))
        #x = F.prelu(self.norm2(self.layer2(x)), 0.5*torch.rand(1, dtype=torch.float32, device=device))
        #x = F.relu(self.layer1(x))
        #x = F.relu(self.layer2(x))
        x = self.layer1(x)
        x = self.active1(x)
        x = self.layer2(x)
        x = self.active2(x)
        return self.output_layer(x)

# class DQN(nn.Module):

#     def __init__(self, n_observations, n_actions):
#         super(DQN, self).__init__()
#         self.layer1 = nn.Linear(n_observations, 16)
#         self.layer2 = nn.Linear(16, 32)
#         self.layer3 = nn.Linear(32, 64)                # Prueba
#         self.layer4 = nn.Linear(64, 64)                # Prueba
#         self.output_layer = nn.Linear(64, n_actions)
#     def forward(self, x):
#         x = F.prelu(self.layer1(x), 0.5*torch.rand(1, dtype=torch.float32, device=device))
#         x = F.prelu(self.layer2(x), 0.5*torch.rand(1, dtype=torch.float32, device=device))
#         x = F.prelu(self.layer3(x), 0.5*torch.rand(1, dtype=torch.float32, device=device))
#         x = F.prelu(self.layer4(x), 0.5*torch.rand(1, dtype=torch.float32, device=device))
#         #x = F.tanh(self.layer4(x))
#         return self.output_layer(x)
#         #return self.output_layer(x)*2

    

        
# class DQN(nn.Module):
#     def __init__(self, n_observations, n_actions):
#         super(DQN, self).__init__()
#         self.layer1 = nn.Linear(n_observations, 16)
#         self.layer2 = nn.Linear(16, 32)
#         self.layer3 = nn.Linear(32, 64)
#         self.layer4 = nn.Linear(64, 64)
#         self.output_layer = nn.Linear(64, n_actions)

#     def forward(self, x):
#         x = F.relu(self.layer1(x))
#         x = F.relu(self.layer2(x))
#         x = F.relu(self.layer3(x))
#         x = F.relu(self.layer4(x))
#         return self.output_layer(x)




## Training

In [5]:
BATCH_SIZE = 128    # number of transitions sampled from the replay buffer
GAMMA = 0.99        # discount factor as mentioned in the previous section
EPS_START = 0.9     # starting value of epsilon
EPS_END = 0.05      # final value of epsilon
EPS_DECAY = 1000    # controls the rate of exponential decay of epsilon, higher means a slower decay
TAU = 0.005         # update rate of the target network
LR = 1e-4           # learning rate of the ``AdamW`` optimizer

# ****************************************************************************
# ****************************************************************************

n_actions = 1

saved_checkpoint = False        # Si se encuentra algun checkpoint guardado y se desea cargar

target_angle = 120
changing_target = False
targets_options = np.array([0, 30, 60, 90, 120, 150, 180, -150, -120, -90, -60, -30])

# ****************************************************************************
if torch.cuda.is_available():
    #num_episodes = 1000
    num_episodes = 100
else:
    #num_episodes = 50
    num_episodes = 10
# ****************************************************************************

state, info = env.reset()
n_observations = len(state)

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)

if saved_checkpoint:
    checkpoint = torch.load("checkpoints/Pendulum_checkpoint_#.tar") # Cambiar el # por el episodio de checkpoint guardado
    start_episode = checkpoint['episodes']
    policy_net.load_state_dict(checkpoint['model_state_dict'])
    target_net.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    target_net.load_state_dict(policy_net.state_dict())
    start_episode = 0

# ****************************************************************************
# ****************************************************************************

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    # *************************************************************************
    with torch.no_grad():
        value = policy_net(state).item()
    if sample > eps_threshold:
        newvalue = value
    else:
        noise = np.random.normal(0, 0.3, size=None)
        newvalue = value + noise
        #newvalue = env.action_space.sample().item()
        
    # *************************************************************************
    if newvalue == 0.0:
        return torch.tensor([0.0], dtype=torch.float32, device=device)
    else:
        #return torch.tensor([np.clip(newvalue, -2.0, 2.0)], dtype=torch.float32, device=device)
        return torch.tensor([newvalue], dtype=torch.float32, device=device)
        #return [newvalue] 
        
episode_rewards = []


def plot_rewards(show_result=False):
    plt.figure(1)
    rewards_t = torch.tensor(episode_rewards, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Time steps (gray lines=episode)')
    plt.ylabel('Reward')
    #plt.plot(rewards_t.numpy(), linewidth=0.2)                 # Cada valor de reward
    # Take 100 episode averages and plot them too
    if len(rewards_t) >= 100:
        means = rewards_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())                                 # Promedios del reward

    # Añadir líneas verticales grises en cada t=200
    #for t in range(200, len(rewards_t) + 1, 200):
    #    plt.axvline(x=t, color='gray', linewidth=0.25)

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

### Training Loop

In [7]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    #action_batch = torch.tensor(np.concatenate(batch.action))
    reward_batch = torch.cat(batch.reward)



    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    
    #state_action_values = policy_net(state_batch) * action_batch.unsqueeze(1)
    #state_action_values = policy_net(state_batch).gather(1, action_batch)
    #state_action_values = policy_net(state_batch).gather(1, action_batch.long().unsqueeze(1))


    state_action_values = policy_net(state_batch)


    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [8]:
new_rewards = []
epoch = 0
epoch_save_checkspoints = num_episodes//5
target_step = num_episodes//len(targets_options)
i_target = 0
u_option = 0

for i_episode in range(start_episode, num_episodes):

    ## *********************************CAMBIO DEL TARGET_ANGLE****************************************
    if changing_target and (i_target == target_step):
        target_angle = targets_options[u_option]
        u_option += 1
        i_target = 1
        print("Target angle >> ", target_angle)
    else:
        i_target += 1

    ## ************************************************************************************************

    # Initialize the environment and get its state
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    print("Episode >> ", i_episode)
    for t in count():
        action = select_action(state)
        #action = np.array(select_action(state), dtype=np.float32)

        #action_step = np.array(action, dtype=np.float32)[0]
        action_step = action.cpu().detach().numpy()

        observation, reward, terminated, truncated, _ = env.step(action_step)

        # ///////////////////////////////////////////////////////////
        #reward = torch.tensor([reward], device=device) # Pendulum original target: theta=0°
        reward = calculate_reward(observation, action.item(), math.radians(target_angle))

        # ///////////////////////////////////////////////////////////

        done = terminated or truncated
        #done = terminated or truncated or (np.absolute(action.item())>2)

        ## *********************************************************************************
        pole_angle = math.degrees(math.atan2(observation[1],observation[0]))

        #vala = state[0].cpu().numpy()
        #print(vala)
        print(np.array([pole_angle, action.item(), reward.item()]))
        new_rewards.append(reward.item())
        ## *********************************************************************************



        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_rewards = episode_rewards + new_rewards
            new_rewards = []
            plot_rewards()
            break

    ## ***********************************************SAVE CHECKPOINTS*********************************************
    if epoch == epoch_save_checkspoints:
        epoch = 1
        checkpoint = {
        'episodes': i_episode,
        'model_state_dict': policy_net_state_dict,
        'optimizer_state_dict': target_net_state_dict,
        # Add other relevant information if needed
        }
        torch.save(checkpoint, f"checkpoints/Pendulum_checkpoint_{i_episode}.tar")
    else:
        epoch += 1
        

## ***********************************************SAVE & LOAD models*******************************************
# Guardar el modelo
torch.save(policy_net.state_dict(), f"models/Pendulum_{num_episodes}eps.pth")
# Cargar el modelo
#model = TheModelClass(*args, **kwargs)
#model.load_state_dict(torch.load(PATH))
#model.eval()
## ************************************************************************************************************

print('Complete')
plot_rewards(show_result=True)
plt.ioff()
plt.show()
env.close()

Episode >>  15
[-1.96216705e+01 -1.67322156e+03 -2.80134756e+06]
[-2.25716296e+01 -2.53084180e+03 -6.40769534e+06]
[-2.71900664e+01 -3.61250024e+03 -1.30537754e+07]
[-3.36310745e+01 -4.81771191e+03 -2.32151715e+07]
[-4.20988896e+01 -6.15470801e+03 -3.78865922e+07]
[-5.28392156e+01 -7.63623193e+03 -5.83196829e+07]
[-6.61187475e+01 -9.29740234e+03 -8.64509984e+07]
[-8.21850353e+01 -1.11630303e+04 -1.24624422e+08]
[-1.01198951e+02 -1.32346455e+04 -1.75169094e+08]
[-1.23139938e+02 -1.54715889e+04 -2.39385556e+08]
[-1.46058249e+02 -1.77646367e+04 -3.15600108e+08]
[-1.68976561e+02 -1.86448926e+04 -3.47650694e+08]
[ 1.68105127e+02 -1.88388242e+04 -3.54920142e+08]
[ 1.45186815e+02 -1.90695625e+04 -3.63667288e+08]
[ 1.22612397e+02 -1.93050664e+04 -3.72704897e+08]
[ 1.00953991e+02 -1.92413945e+04 -3.70250509e+08]
[ 8.05055136e+01 -1.86751504e+04 -3.48779921e+08]
[ 6.12764940e+01 -1.78097773e+04 -3.17205982e+08]
[ 4.30364429e+01 -1.68684766e+04 -2.84562374e+08]
[ 2.53754325e+01 -1.60515596e+04 -2

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

### Version anterior de optimize_model()

In [ ]:
# def optimize_model():
#     if len(memory) < BATCH_SIZE:
#         return
#     transitions = memory.sample(BATCH_SIZE)
#     # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
#     # detailed explanation). This converts batch-array of Transitions
#     # to Transition of batch-arrays.
#     batch = Transition(*zip(*transitions))

#     # Compute a mask of non-final states and concatenate the batch elements
#     # (a final state would've been the one after which simulation ended)
#     non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
#                                           batch.next_state)), device=device, dtype=torch.bool)
#     non_final_next_states = torch.cat([s for s in batch.next_state
#                                                 if s is not None])
#     state_batch = torch.cat(batch.state)
#     #action_batch = torch.cat(batch.action)
#     action_batch = torch.tensor(np.concatenate(batch.action))
#     reward_batch = torch.cat(batch.reward)

#     # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
#     # columns of actions taken. These are the actions which would've been taken
#     # for each batch state according to policy_net
    
#     state_action_values = policy_net(state_batch)

#     # Compute V(s_{t+1}) for all next states.
#     # Expected values of actions for non_final_next_states are computed based
#     # on the "older" target_net; selecting their best reward with max(1).values
#     # This is merged based on the mask, such that we'll have either the expected
#     # state value or 0 in case the state was final.
#     next_state_values = torch.zeros(BATCH_SIZE, device=device)
#     with torch.no_grad():
#         next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
#     # Compute the expected Q values
#     expected_state_action_values = (next_state_values * GAMMA) + reward_batch

#     # Compute Huber loss
#     criterion = nn.SmoothL1Loss()
#     loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

#     # Optimize the model
#     optimizer.zero_grad()
#     loss.backward()
#     # In-place gradient clipping
#     torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
#     optimizer.step()

In [ ]:
# val = 25
# valores = np.linspace(-3, 3, val)

val = 41
valores = np.linspace(-5, 5, val)

for i in range(val):
    print([valores[i], np.minimum(2-np.absolute(valores[i]),0)])
    #print(valores[i])
    print(8 - np.minimum(2-np.absolute(valores[i]),0))






#print(np.min(2-np.absolute(torque),0))